<a href="https://colab.research.google.com/github/davidclizbe/datascience/blob/master/ClizbeWeatherRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weatherinszeged = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [ ]:
weatherinszeged

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
...,...,...,...,...,...,...,...,...,...,...,...,...
96448,2016-09-09 17:00:00+00:00,Partly Cloudy,rain,26.016667,26.016667,0.43,10.9963,31.0,16.1000,0.0,1014.36,Partly cloudy starting in the morning.
96449,2016-09-09 18:00:00+00:00,Partly Cloudy,rain,24.583333,24.583333,0.48,10.0947,20.0,15.5526,0.0,1015.16,Partly cloudy starting in the morning.
96450,2016-09-09 19:00:00+00:00,Partly Cloudy,rain,22.038889,22.038889,0.56,8.9838,30.0,16.1000,0.0,1015.66,Partly cloudy starting in the morning.
96451,2016-09-09 20:00:00+00:00,Partly Cloudy,rain,21.522222,21.522222,0.60,10.5294,20.0,16.1000,0.0,1015.95,Partly cloudy starting in the morning.


In [ ]:
weatherinszeged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   date                 96453 non-null  datetime64[ns, UTC]
 1   summary              96453 non-null  object             
 2   preciptype           96453 non-null  object             
 3   temperature          96453 non-null  float64            
 4   apparenttemperature  96453 non-null  float64            
 5   humidity             96453 non-null  float64            
 6   windspeed            96453 non-null  float64            
 7   windbearing          96453 non-null  float64            
 8   visibility           96453 non-null  float64            
 9   loudcover            96453 non-null  float64            
 10  pressure             96453 non-null  float64            
 11  dailysummary         96453 non-null  object             
dtypes: datetime64[ns, 

In [ ]:
weatherinszeged.describe()

,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure
count,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.0,96453.000000
mean,11.932678,10.855029,0.734899,10.810640,187.509232,10.347325,0.0,1003.235956
std,9.551546,10.696847,0.195473,6.913571,107.383428,4.192123,0.0,116.969906
min,-21.822222,-27.716667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,4.688889,2.311111,0.600000,5.828200,116.000000,8.339800,0.0,1011.900000
50%,12.000000,12.000000,0.780000,9.965900,180.000000,10.046400,0.0,1016.450000
75%,18.838889,18.838889,0.890000,14.135800,290.000000,14.812000,0.0,1021.090000
max,39.905556,39.344444,1.000000,63.852600,359.000000,16.100000,0.0,1046.380000


In [ ]:
weatherinszeged.corr()

,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure
temperature,1.000000,0.992629,-0.632255,0.008957,0.029988,0.392847,NaN,-0.005447
apparenttemperature,0.992629,1.000000,-0.602571,-0.056650,0.029031,0.381718,NaN,-0.000219
humidity,-0.632255,-0.602571,1.000000,-0.224951,0.000735,-0.369173,NaN,0.005454
windspeed,0.008957,-0.056650,-0.224951,1.000000,0.103822,0.100749,NaN,-0.049263
windbearing,0.029988,0.029031,0.000735,0.103822,1.000000,0.047594,NaN,-0.011651
visibility,0.392847,0.381718,-0.369173,0.100749,0.047594,1.000000,NaN,0.059818
loudcover,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pressure,-0.005447,-0.000219,0.005454,-0.049263,-0.011651,0.059818,NaN,1.000000


In [ ]:
weather = pd.get_dummies(weatherinszeged, dummy_na=True)
weather.head()

,date,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,summary_Breezy,summary_Breezy and Dry,summary_Breezy and Foggy,summary_Breezy and Mostly Cloudy,summary_Breezy and Overcast,summary_Breezy and Partly Cloudy,summary_Clear,summary_Dangerously Windy and Partly Cloudy,summary_Drizzle,summary_Dry,summary_Dry and Mostly Cloudy,summary_Dry and Partly Cloudy,summary_Foggy,summary_Humid and Mostly Cloudy,summary_Humid and Overcast,summary_Humid and Partly Cloudy,summary_Light Rain,summary_Mostly Cloudy,summary_Overcast,summary_Partly Cloudy,summary_Rain,summary_Windy,summary_Windy and Dry,summary_Windy and Foggy,summary_Windy and Mostly Cloudy,summary_Windy and Overcast,summary_Windy and Partly Cloudy,summary_nan,preciptype_null,preciptype_rain,preciptype_snow,...,dailysummary_Partly cloudy starting overnight continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the evening.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening and windy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until morning and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until morning.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the afternoon.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until night and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until night.,dailysummary_Partly cloudy starting overnight.,dailysummary_Partly cloudy throughout the day and breezy in the afternoon.,dailysummary_Partly cloudy throughout the day and breezy in the evening.,dailysummary_Partly cloudy throughout the day and breezy in the morning.,dailysummary_Partly cloudy throughout the day and breezy starting in the afternoon continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until night.,dailysummary_Partly cloudy throughout the day and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day.,dailysummary_Partly cloudy until afternoon.,dailysummary_Partly cloudy until evening and breezy in the afternoon.,dailysummary_Partly cloudy until evening and breezy in the morning.,dailysummary_Partly cloudy until evening and breezy overnight.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy until evening.,dailysummary_Partly cloudy until morning.,dailysummary_Partly cloudy until night and breezy in the afternoon.,dailysummary_Partly cloudy until night and breezy in the morning.,dailysummary_Partly cloudy until night and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until night.,dailysummary_Rain throughout the day.,dailysummary_Rain until afternoon.,dailysummary_Rain until morning.,dailysummary_Windy in the afternoon.,dailysummary_nan
0,2006-03-31 22:00:00

In [ ]:
weather.dropna(inplace=True)
weather.head()

,date,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,summary_Breezy,summary_Breezy and Dry,summary_Breezy and Foggy,summary_Breezy and Mostly Cloudy,summary_Breezy and Overcast,summary_Breezy and Partly Cloudy,summary_Clear,summary_Dangerously Windy and Partly Cloudy,summary_Drizzle,summary_Dry,summary_Dry and Mostly Cloudy,summary_Dry and Partly Cloudy,summary_Foggy,summary_Humid and Mostly Cloudy,summary_Humid and Overcast,summary_Humid and Partly Cloudy,summary_Light Rain,summary_Mostly Cloudy,summary_Overcast,summary_Partly Cloudy,summary_Rain,summary_Windy,summary_Windy and Dry,summary_Windy and Foggy,summary_Windy and Mostly Cloudy,summary_Windy and Overcast,summary_Windy and Partly Cloudy,summary_nan,preciptype_null,preciptype_rain,preciptype_snow,...,dailysummary_Partly cloudy starting overnight continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the evening.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening and windy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until morning and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until morning.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the afternoon.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until night and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until night.,dailysummary_Partly cloudy starting overnight.,dailysummary_Partly cloudy throughout the day and breezy in the afternoon.,dailysummary_Partly cloudy throughout the day and breezy in the evening.,dailysummary_Partly cloudy throughout the day and breezy in the morning.,dailysummary_Partly cloudy throughout the day and breezy starting in the afternoon continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until night.,dailysummary_Partly cloudy throughout the day and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day.,dailysummary_Partly cloudy until afternoon.,dailysummary_Partly cloudy until evening and breezy in the afternoon.,dailysummary_Partly cloudy until evening and breezy in the morning.,dailysummary_Partly cloudy until evening and breezy overnight.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy until evening.,dailysummary_Partly cloudy until morning.,dailysummary_Partly cloudy until night and breezy in the afternoon.,dailysummary_Partly cloudy until night and breezy in the morning.,dailysummary_Partly cloudy until night and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until night.,dailysummary_Rain throughout the day.,dailysummary_Rain until afternoon.,dailysummary_Rain until morning.,dailysummary_Windy in the afternoon.,dailysummary_nan
0,2006-03-31 22:00:00

In [ ]:
y = weather['temperature']
y.head()

0    9.472222
1    9.355556
2    9.377778
3    8.288889
4    8.755556
Name: temperature, dtype: float64

In [ ]:
X = weather.drop(['temperature'], axis= 1)
X.head()

,date,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,summary_Breezy,summary_Breezy and Dry,summary_Breezy and Foggy,summary_Breezy and Mostly Cloudy,summary_Breezy and Overcast,summary_Breezy and Partly Cloudy,summary_Clear,summary_Dangerously Windy and Partly Cloudy,summary_Drizzle,summary_Dry,summary_Dry and Mostly Cloudy,summary_Dry and Partly Cloudy,summary_Foggy,summary_Humid and Mostly Cloudy,summary_Humid and Overcast,summary_Humid and Partly Cloudy,summary_Light Rain,summary_Mostly Cloudy,summary_Overcast,summary_Partly Cloudy,summary_Rain,summary_Windy,summary_Windy and Dry,summary_Windy and Foggy,summary_Windy and Mostly Cloudy,summary_Windy and Overcast,summary_Windy and Partly Cloudy,summary_nan,preciptype_null,preciptype_rain,preciptype_snow,preciptype_nan,...,dailysummary_Partly cloudy starting overnight continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the evening.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening and windy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until morning and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until morning.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the afternoon.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until night and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until night.,dailysummary_Partly cloudy starting overnight.,dailysummary_Partly cloudy throughout the day and breezy in the afternoon.,dailysummary_Partly cloudy throughout the day and breezy in the evening.,dailysummary_Partly cloudy throughout the day and breezy in the morning.,dailysummary_Partly cloudy throughout the day and breezy starting in the afternoon continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until night.,dailysummary_Partly cloudy throughout the day and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day.,dailysummary_Partly cloudy until afternoon.,dailysummary_Partly cloudy until evening and breezy in the afternoon.,dailysummary_Partly cloudy until evening and breezy in the morning.,dailysummary_Partly cloudy until evening and breezy overnight.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy until evening.,dailysummary_Partly cloudy until morning.,dailysummary_Partly cloudy until night and breezy in the afternoon.,dailysummary_Partly cloudy until night and breezy in the morning.,dailysummary_Partly cloudy until night and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until night.,dailysummary_Rain throughout the day.,dailysummary_Rain until afternoon.,dailysummary_Rain until morning.,dailysummary_Windy in the afternoon.,dailysummary_nan
0,2006-03-31 22:00

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

,date,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,summary_Breezy,summary_Breezy and Dry,summary_Breezy and Foggy,summary_Breezy and Mostly Cloudy,summary_Breezy and Overcast,summary_Breezy and Partly Cloudy,summary_Clear,summary_Dangerously Windy and Partly Cloudy,summary_Drizzle,summary_Dry,summary_Dry and Mostly Cloudy,summary_Dry and Partly Cloudy,summary_Foggy,summary_Humid and Mostly Cloudy,summary_Humid and Overcast,summary_Humid and Partly Cloudy,summary_Light Rain,summary_Mostly Cloudy,summary_Overcast,summary_Partly Cloudy,summary_Rain,summary_Windy,summary_Windy and Dry,summary_Windy and Foggy,summary_Windy and Mostly Cloudy,summary_Windy and Overcast,summary_Windy and Partly Cloudy,summary_nan,preciptype_null,preciptype_rain,preciptype_snow,preciptype_nan,...,dailysummary_Partly cloudy starting overnight continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the evening.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening and windy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until morning and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until morning.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the afternoon.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until night and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until night.,dailysummary_Partly cloudy starting overnight.,dailysummary_Partly cloudy throughout the day and breezy in the afternoon.,dailysummary_Partly cloudy throughout the day and breezy in the evening.,dailysummary_Partly cloudy throughout the day and breezy in the morning.,dailysummary_Partly cloudy throughout the day and breezy starting in the afternoon continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until night.,dailysummary_Partly cloudy throughout the day and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day.,dailysummary_Partly cloudy until afternoon.,dailysummary_Partly cloudy until evening and breezy in the afternoon.,dailysummary_Partly cloudy until evening and breezy in the morning.,dailysummary_Partly cloudy until evening and breezy overnight.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy until evening.,dailysummary_Partly cloudy until morning.,dailysummary_Partly cloudy until night and breezy in the afternoon.,dailysummary_Partly cloudy until night and breezy in the morning.,dailysummary_Partly cloudy until night and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until night.,dailysummary_Rain throughout the day.,dailysummary_Rain until afternoon.,dailysummary_Rain until morning.,dailysummary_Windy in the afternoon.,dailysummary_nan
72148,2014-12-03 0

In [ ]:
import statsmodels.api as sm
import numpy as np

model = sm.OLS(y_train,X_train)
results = model.fit()
results.params

ValueError: ignored